In [111]:
!pip install -q -U pip langchain tiktoken chromadb
!pip install -q langchain==0.0.332
# !pip install -q chromadb==0.4.10

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.8.41 requires urllib3<2, but you have urllib3 2.2.1 which is incompatible.


In [112]:
!pip show langchain

Name: langchain
Version: 0.0.332
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /home/mesabah/.local/lib/python3.10/site-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: llama-index


In [1]:
import os
import pandas as pd
from pathlib import Path
from langchain.embeddings import OpenAIEmbeddings    
from langchain.vectorstores import Chroma
from langchain.globals import set_llm_cache
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain.cache import InMemoryCache
from langchain.llms import OpenAI
from tqdm import tqdm
import json

In [2]:
os.environ["OPENAI_API_KEY"] = 'sk-'

## load the splits

In [3]:
api_list = pd.read_csv("APIs List.csv")

In [4]:
API_to_Splits = {}
for _, row in api_list.iterrows():
    print("Load Files of "+row['Name']+"API ................")
    loader = DirectoryLoader(
        'data/'+row['Name']+'/'+row['API_Ref_File'][:-4]+"/",
        glob="**/*.txt",
        loader_cls=TextLoader,
        show_progress=True
    )
    docs = loader.load()
    API_to_Splits[row['Name']] = docs

Load Files of PyDrive2API ................


100%|██████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1380.80it/s]


Load Files of pytensorAPI ................


100%|████████████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 2198.03it/s]


Load Files of ydata-profilingAPI ................


...

100%|██████████████████████████████████████████████████████████████████████████████████| 289/289 [00:00<00:00, 2411.39it/s]


Load Files of scikit-learnAPI ................


100%|██████████████████████████████████████████████████████████████████████████████████| 817/817 [00:00<00:00, 2439.47it/s]


Load Files of pandasAPI ................


100%|██████████████████████████████████████████████████████████████████████████████████| 108/108 [00:00<00:00, 2187.61it/s]


## Embedding and Storing

In [3]:
# cach prompts
set_llm_cache(InMemoryCache())

In [7]:
for api in tqdm(API_to_Splits):
    from langchain.vectorstores import Chroma   
    Chroma().delete_collection()
    vectorstore = Chroma.from_documents(
                    documents=API_to_Splits[api],
                    embedding=OpenAIEmbeddings(disallowed_special=()),
                    persist_directory="./chroma_db/"+api
                    ) 

100%|██████████████████████████████████████████████████████████████████████████████████████| 50/50 [08:30<00:00, 10.22s/it]


## Retrieving

In [3]:
def retrieve_splits(instant):
    instruction = instant["instruction"]
    api = instant["api"]
    set_llm_cache(InMemoryCache())
    vectorstore = Chroma(persist_directory="./chroma_db/"+api, embedding_function=OpenAIEmbeddings())
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    retriever_from_llm = MultiQueryRetriever.from_llm(
            retriever=retriever, 
            llm=OpenAI(
                model_name="gpt-3.5-turbo-instruct",
                temperature=0.5
                )
        )
    unique_splits = retriever_from_llm.get_relevant_documents(query=instruction)
    for i, split in enumerate(unique_splits):
        instant["input_"+str(i)] = split.page_content
    return instant

In [4]:
# Get best 3 splits for testing sataset instruction
with open("./pairs_data/cleaned_data/data_test.json", 'r') as file:
    data = json.load(file)
    
data_docu = []
for instant in tqdm(data):
    instant.pop('input', None)
    data_docu.append(retrieve_splits(instant))
    
with open('./pairs_data/cleaned_data/data_splits_docu_test.json', 'w') as f:
    json.dump(data_docu, f)

100%|██████████| 127/127 [07:02<00:00,  3.33s/it]


In [22]:
# Get best 3 splits for training sataset instruction
with open("./pairs_data/cleaned_data/data_train.json", 'r') as file:
    data = json.load(file)
    
data_docu = []
for instant in tqdm(data):
    instant.pop('input', None)
    data_docu.append(retrieve_splits(instant))
    
with open('./pairs_data/cleaned_data/data_splits_docu_train.json', 'w') as f:
    json.dump(data_docu, f)

100%|████████████████████████████████████████████████████████████████████████████████████| 811/811 [30:27<00:00,  2.25s/it]


Combine the split files into one input

In [5]:
with open("./pairs_data/cleaned_data/data_splits_docu_train.json", 'r') as file:
    data = json.load(file)

data_docu = []
for instant in tqdm(data):
    input = ""
    i = 0
    while True:
        key = "input_"+str(i)
        if key in instant.keys():
            input += "\n\n==================\n Document "+str(i)+" \n----------------\n"
            input += instant[key]
            instant.pop(key, None)
        else:
            break
        i+=1
    instant['documentation'] = input
    data_docu.append(instant)

with open('./pairs_data/cleaned_data/data_docu_train.json', 'w') as f:
    json.dump(data_docu, f)

100%|██████████| 811/811 [00:00<00:00, 31130.62it/s]

In [6]:
with open("./pairs_data/cleaned_data/data_splits_docu_test.json", 'r') as file:
    data = json.load(file)

data_docu = []
for instant in tqdm(data):
    input = ""
    i = 0
    while True:
        key = "input_"+str(i)
        if key in instant.keys():
            input += "\n\n==================\n Document "+str(i)+" \n----------------\n"
            input += instant[key]
            instant.pop(key, None)
        else:
            break
        i+=1
    instant['documentation'] = input
    data_docu.append(instant)

with open('./pairs_data/cleaned_data/data_docu_test.json', 'w') as f:
    json.dump(data_docu, f, ensure_ascii=False, indent=4)

100%|██████████| 127/127 [00:00<00:00, 13548.94it/s]